In [1]:
%pip install qiskit
%pip install qiskit-aer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from math import pi
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer


In [3]:
# Fourier transform 
def qft(circ, q, n):
    for i in range(n,0,-1):
        circ.h(q[i-1])

        for j in range(i-1,0,-1):
            circ.cp(2*pi/2**(i-j+1), q[j-1], q[i-1])

# Inverse Fourier transform 
def iqft(circ, q, n):
    for i in range(1,n+1):
        for j in range(1,i):
            circ.cp(-2*pi/2**(i-j+1), q[j-1], q[i-1])

        circ.h(q[i-1])
        
def add(circ, a, b, n):
    n += 1

    qft(circ, b, n)

    for i in range(n,0,-1):
        for j in range(i,0,-1):
            if len(a) - 1 >= j - 1:
                circ.cp(2*pi/2**(i-j+1), a[j-1], b[i-1])

    iqft(circ, b, n)

In [4]:
def qadd_num(first, second, N):
    
    a = QuantumRegister(N)
    b = QuantumRegister(N+1)

    ca = ClassicalRegister(N)
    cb = ClassicalRegister(N+1)

    qc = QuantumCircuit(a, b, ca, cb)

    for i in range(0, N+1):
        if first[N-i] == "1":
            qc.x(a[i])

    for i in range(0, N+1):
        if second[N-i] == "1":
            qc.x(b[i])
    
    add(qc, a, b, N)

    qc.measure(a, ca)
    qc.measure(b, cb)

    qc.draw(filename='./circuit_img.tex')

    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()

    x = result_sim.get_counts(qc)

    return list(x.keys())[0].split(" ")[0]

In [9]:
def find_pair(target, arr):
    n = len(arr)
    for i in range(n):
        for j in range(i+1, n):
            first = f'{arr[i]:07b}'
            second = f'{arr[j]:07b}'
            qsum = qadd_num(first, second, 6)

            if int(qsum, 2) == target:
                return arr[i], arr[j]
    return -1

In [11]:
target = 24
arr = [1,3,5,7,11,13,15]
pair = find_pair (target, arr)
print(f"\n Target Sum {target} can be achieved by sum of : {pair}")


 Target Sum 24 can be achieved by sum of : (11, 13)
